In [8]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder


label_cols = ['attack_cat', 'Label']
date_cols = ['Stime', 'Ltime']
cols = [
    'srcip',
    'sport',
    'dstip',
    'dsport',
    'proto',
    'state',
    'dur',
    'sbytes',
    'dbytes',
    'sttl',
    'dttl',
    'sloss',
    'dloss',
    'service',
    'Sload',
    'Dload',
    'Spkts',
    'Dpkts',
    'swin',
    'dwin',
    'stcpb',
    'dtcpb',
    'smeansz',
    'dmeansz',
    
    'trans_depth',
    'res_bdy_len',
    'Sjit',
    'Djit',
    'Stime',
    'Ltime',
    'Sintpkt',
    'Dintpkt',
    'tcprtt',
    'synack',
    'ackdat',
    'is_sm_ips_ports',
    'ct_state_ttl',
    
#     'ct_flw_http_mthd',
#     'is_ftp_login',
#     'ct_ftp_cmd',
    
    'ct_srv_src',
    'ct_srv_dst',
    
#     'ct_dst_ltm',
#     'ct_src_ltm',
    
    'ct_src_dport_ltm',
    'ct_dst_sport_ltm',
    'ct_dst_src_ltm',
    'attack_cat',
    'Label',
]

def ip_to_int(ip):
    z = ''
    parts = ip.split('.')
    for prt in range(3):
        if len(parts[prt]) == 3:
            z = parts[prt] + z
        elif len(parts[prt]) == 2:
            z = '0' + parts[prt] + z
        else:
            z = '00' + parts[prt] + z
    return int(z)

converters = {
    'srcip': ip_to_int,
    'dstip': ip_to_int,
}

# Read csv
df = pd.read_csv(
    '../data/UNSW-NB15-BALANCED-TRAIN.csv',
    parse_dates=date_cols,
    usecols=cols,
    skipinitialspace=True,
    converters=converters
)

df['sport'] = df['sport'].replace('-', 0)
df['dsport'] = df['dsport'].replace('-', 0)
df['sport'] = df['sport'].apply(lambda x: int(str(x), 16))
df['dsport'] = df['dsport'].apply(lambda x: int(str(x), 16))
df['service'] = df['service'].replace('-', '')


le = LabelEncoder()
df['proto'] = le.fit_transform(df['proto'])
df['state'] = le.fit_transform(df['state'])
df['service'] = le.fit_transform(df['service'])
df['attack_cat'] = le.fit_transform(df['attack_cat'])

df.head()

C:\Users\52111\AppData\Local\Temp\ipykernel_3040\3377561031.py:87: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ackdat,is_sm_ips_ports,ct_state_ttl,ct_srv_src,ct_srv_dst,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,166059,349328,126171149,83,119,2,0.001050,146,178,31,...,0.000000,0,0,3,4,1,1,1,13,0
1,166059,157526,126171149,33,113,5,4.264797,2934,3738,31,...,0.000134,0,0,5,5,5,5,8,13,0
2,176045175,4163,126171149,83,119,6,0.000003,114,0,254,...,0.000000,0,2,17,17,14,14,17,7,1
3,176045175,4163,126171149,83,119,6,0.000003,114,0,254,...,0.000000,0,2,25,25,25,25,25,7,1
4,166059,214130,126171149,66601,113,5,0.052957,3718,43086,31,...,0.000130,0,0,5,8,1,1,2,13,0


In [9]:
df.describe()

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ackdat,is_sm_ips_ports,ct_state_ttl,ct_srv_src,ct_srv_dst,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
count,4.497960e+05,449796.000000,4.497960e+05,4.497960e+05,449796.000000,449796.000000,449796.000000,4.497960e+05,4.497960e+05,449796.000000,...,449796.000000,449796.000000,449796.000000,449796.000000,449796.000000,449796.000000,449796.000000,449796.000000,449796.000000,449796.000000
mean,9.646951e+07,166478.437687,1.284399e+08,4.295583e+04,114.453261,5.085528,0.694259,5.092681e+03,2.277625e+04,138.566168,...,0.006775,0.000972,0.931353,13.858387,13.683721,8.416918,6.496405,12.401215,9.646331,0.500000
std,8.570929e+07,139797.213314,1.231686e+07,1.140547e+06,12.583730,1.293619,13.697272,1.157077e+05,1.449404e+05,109.317462,...,0.026773,0.031155,0.982520,13.818018,13.880075,11.033970,7.860205,14.575122,3.540252,0.500001
min,1.660590e+05,0.000000,2.240000e+02,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,1.660590e+05,4163.000000,1.261711e+08,8.300000e+01,113.000000,5.000000,0.000008,1.140000e+02,0.000000e+00,31.000000,...,0.000000,0.000000,0.000000,3.000000,3.000000,1.000000,1.000000,1.000000,7.000000,0.000000
50%,1.760452e+08,161571.000000,1.261711e+08,8.300000e+01,119.000000,5.000000,0.001058,2.640000e+02,1.780000e+02,62.000000,...,0.000000,0.000000,1.000000,7.000000,7.000000,2.000000,1.000000,4.000000,12.500000,0.500000
75%,1.760452e+08,291897.000000,1.261711e+08,5.923000e+03,119.000000,6.000000,0.098976,2.054000e+03,3.276000e+03,254.000000,...,0.000136,0.000000,2.000000,24.000000,24.000000,16.000000,13.000000,24.000000,13.000000,1.000000
max,2.411682e+08,415029.000000,2.411682e+08,5.389893e+08,132.000000,12.000000,8760.777344,1.435577e+07,1.465753e+07,255.000000,...,3.551121,1.000000,6.000000,67.000000,67.000000,67.000000,60.000000,67.000000,13.000000,1.000000


In [10]:
df.values

array([[166059, 349328, 126171149, ..., 1, 13, 0],
       [166059, 157526, 126171149, ..., 8, 13, 0],
       [176045175, 4163, 126171149, ..., 17, 7, 1],
       ...,
       [166059, 201106, 126171149, ..., 1, 13, 0],
       [176045175, 4163, 126171149, ..., 41, 7, 1],
       [166059, 354344, 126171149, ..., 1, 13, 0]], dtype=object)

In [11]:
# Correlation / Co-variance
threshold = 0.95

# Absolute value correlation matrix
corr_matrix = df.drop(columns=label_cols).corr().abs()
corr_matrix.head()

C:\Users\52111\AppData\Local\Temp\ipykernel_3040\3278412854.py:5: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = df.drop(columns=label_cols).corr().abs()


,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_srv_src,ct_srv_dst,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm
srcip,1.000000,0.221771,0.070231,0.033644,0.015647,0.629844,0.009991,0.005832,0.144180,0.906528,...,0.230102,0.203657,0.230866,0.028966,0.880090,0.532019,0.533333,0.539218,0.582074,0.596441
sport,0.221771,1.000000,0.021531,0.007765,0.141644,0.185740,0.013746,0.008924,0.039544,0.221036,...,0.092319,0.085087,0.088912,0.037137,0.238090,0.146677,0.149100,0.161528,0.204456,0.163564
dstip,0.070231,0.021531,1.000000,0.008053,0.075822,0.119629,0.041925,0.009608,0.029075,0.125955,...,0.047511,0.043069,0.046550,0.120622,0.166834,0.239521,0.239491,0.234670,0.250091,0.243959
dsport,0.033644,0.007765,0.008053,1.000000,0.017029,0.008417,0.001377,0.000661,0.001302,0.030425,...,0.005064,0.004458,0.005107,0.001175,0.028505,0.019720,0.020523,0.024648,0.025683,0.025788
proto,0.015647,0.141644,0.075822,0.017029,1.000000,0.074457,0.067794,0.006401,0.018060,0.016605,...,0.029834,0.027051,0.029223,0.268768,0.022091,0.231564,0.232181,0.212548,0.216749,0.220183


In [12]:
# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
upper.head()

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_srv_src,ct_srv_dst,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm
srcip,NaN,0.221771,0.070231,0.033644,0.015647,0.629844,0.009991,0.005832,0.144180,0.906528,...,0.230102,0.203657,0.230866,0.028966,0.880090,0.532019,0.533333,0.539218,0.582074,0.596441
sport,NaN,NaN,0.021531,0.007765,0.141644,0.185740,0.013746,0.008924,0.039544,0.221036,...,0.092319,0.085087,0.088912,0.037137,0.238090,0.146677,0.149100,0.161528,0.204456,0.163564
dstip,NaN,NaN,NaN,0.008053,0.075822,0.119629,0.041925,0.009608,0.029075,0.125955,...,0.047511,0.043069,0.046550,0.120622,0.166834,0.239521,0.239491,0.234670,0.250091,0.243959
dsport,NaN,NaN,NaN,NaN,0.017029,0.008417,0.001377,0.000661,0.001302,0.030425,...,0.005064,0.004458,0.005107,0.001175,0.028505,0.019720,0.020523,0.024648,0.025683,0.025788
proto,NaN,NaN,NaN,NaN,NaN,0.074457,0.067794,0.006401,0.018060,0.016605,...,0.029834,0.027051,0.029223,0.268768,0.022091,0.231564,0.232181,0.212548,0.216749,0.220183


In [13]:
# Drop duplicate columns by finding their correlations between each other
to_drop_raw = []

# Column for column in upper.columns if any upper[column] > threshold
for column in upper.columns:
    for index, correlation in enumerate(upper[column]):
        if (correlation > threshold):
            to_drop_raw.append(upper.columns[index])
            
to_drop = list(set(to_drop_raw))
to_drop

['sbytes', 'dloss', 'ct_srv_dst', 'swin', 'dbytes', 'ct_srv_src']

In [14]:
# Drop label columns and duplicate columns

drop_cols = label_cols + to_drop

X = df.drop(columns=drop_cols)
y = df['Label']

X.columns

Index(['srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'dur', 'sttl',
       'dttl', 'sloss', 'service', 'Sload', 'Dload', 'Spkts', 'Dpkts', 'dwin',
       'stcpb', 'dtcpb', 'smeansz', 'dmeansz', 'trans_depth', 'res_bdy_len',
       'Sjit', 'Djit', 'Stime', 'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt',
       'synack', 'ackdat', 'is_sm_ips_ports', 'ct_state_ttl',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm'],
      dtype='object')

In [15]:
# X_train_validation includes training set and validation set
# X_test includes test set
# y_train_validation includes the labels for both the training set and validation set
# y_test includes the labels for the test set
X_train_validation, X_test, y_train_validation, y_test = train_test_split(X, y, test_size=0.2)

X_train_validation.head()

,srcip,sport,dstip,dsport,proto,state,dur,sttl,dttl,sloss,...,Sintpkt,Dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm
327410,166059,29473,126171149,128,113,5,1.074957,31,29,3,...,82.578156,63.204000,0.000610,0.000485,0.000125,0,0,1,1,2
241166,176045175,291897,126171149,83,119,6,0.000006,254,0,0,...,0.006000,0.000000,0.000000,0.000000,0.000000,0,2,18,18,18
118839,166059,137090,126171149,26753,113,5,0.018217,31,29,4,...,1.193800,1.044706,0.000567,0.000453,0.000114,0,0,3,1,3
193485,166059,35204,126171149,342417,113,5,0.361264,31,29,1,...,51.562287,32.787818,0.000748,0.000593,0.000155,0,0,1,1,2
312446,166059,349593,126171149,273,119,2,0.427385,31,29,0,...,137.343672,126.321000,0.000000,0.000000,0.000000,0,0,1,1,3


In [ ]:
# X_train is the training set
# X_validation is the validation set
# y_train is the labels for the training set
# y_validation is the labels for the validation set
X_train, X_validation, y_train, y_validation = train_test_split(X_train_validation, y_train_validation, test_size=0.2)

X_train.head()

In [243]:
# DTree
# model = DecisionTreeClassifier()

# KNN - K-Nearest Neighbours Classifier
model = KNeighborsClassifier(n_neighbors=5)

# Train
model.fit(X_train, y_train)



KNeighborsClassifier()

In [247]:
from sklearn.metrics import classification_report, confusion_matrix

# Validate model
predictions = model.predict(X_validation)

print(classification_report(y_validation, predictions))
print(confusion_matrix(y_validation, predictions))


# score = accuracy_score(y_validation, predictions)


              precision    recall  f1-score   support

           0       1.00      0.98      0.99     35990
           1       0.98      1.00      0.99     35978

    accuracy                           0.99     71968
   macro avg       0.99      0.99      0.99     71968
weighted avg       0.99      0.99      0.99     71968

[[35381   609]
 [   56 35922]]
